In [1]:
data_dir = 'E:/Jupyter NB/10 class Classification/'

train_dir = 'E:/Jupyter NB/10 class Classification/Train_val_test/train'
val_dir = 'E:/Jupyter NB/10 class Classification/Train_val_test/val'
test_dir = 'E:/Jupyter NB/10 class Classification/Train_val_test/test'

In [2]:
import numpy as np
import sys
sys.path.append('utils/')

import keras
from keras import layers
from keras import models

from keras import optimizers
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

from image_preprocessing_ver2 import ImageDataGenerator

from keras.models import Model
from keras.layers import Lambda, concatenate, Activation
from keras.losses import categorical_crossentropy as logloss
from keras.metrics import categorical_accuracy, top_k_categorical_accuracy
from keras import backend as K

Using TensorFlow backend.


In [3]:
train_logits = np.load(data_dir + 'train_logits.npy', allow_pickle = True)[()]
val_logits = np.load(data_dir + 'val_logits.npy', allow_pickle = True)[()]

In [4]:
def preprocess_input(x):
    x /= 255.0
    x -= 0.5
    x *= 2.0
    return x

In [5]:

data_generator = ImageDataGenerator(
    data_format='channels_last',
    preprocessing_function=preprocess_input
)

# note: i'm also passing dicts of logits
train_generator = data_generator.flow_from_directory(
    train_dir, train_logits,
    target_size=(299, 299),
    batch_size=20
)

validation_generator = data_generator.flow_from_directory(
    val_dir, val_logits,
    target_size=(299, 299),
    batch_size=20
)

Found 2000 images belonging to 20 classes.
Found 400 images belonging to 20 classes.


In [6]:
std_model = models.Sequential([
    layers.Conv2D(16, (3,3), activation = 'relu', input_shape = (299,299,3)),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(32, (3,3), activation = 'relu'),
    layers.MaxPooling2D(2,2),
    
    layers.Flatten(),
    layers.Dense(64, activation = 'relu'),
    layers.Dense(20, activation = 'softmax')
    
])

std_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 297, 297, 16)      448       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 148, 148, 16)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 146, 146, 32)      4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 73, 73, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 170528)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                10913856  
_________________________________________________________________
dense_2 (Dense)              (None, 20)               

In [7]:
temperature = 5.0

# remove softmax
std_model.layers.pop()

# usual probabilities
logits = std_model.layers[-1].output
probabilities = Activation('softmax')(logits)

# softed probabilities
logits_T = Lambda(lambda x: x/temperature)(logits)
probabilities_T = Activation('softmax')(logits_T)

output = concatenate([probabilities, probabilities_T])
model = Model(std_model.input, output)
# now model outputs 512 dimensional vectors

In [8]:
def knowledge_distillation_loss(y_true, y_pred, lambda_const):    
    
    # split in 
    #    onehot hard true targets
    #    logits from xception
    y_true, logits = y_true[:, :20], y_true[:, 20:]
    
    # convert logits to soft targets
    y_soft = K.softmax(logits/temperature)
    
    # split in 
    #    usual output probabilities
    #    probabilities made softer with temperature
    y_pred, y_pred_soft = y_pred[:, :20], y_pred[:, 20:]    
    
    return lambda_const*logloss(y_true, y_pred) + logloss(y_soft, y_pred_soft)

In [9]:
def accuracy(y_true, y_pred):
    y_true = y_true[:, :20]
    y_pred = y_pred[:, :20]
    return categorical_accuracy(y_true, y_pred)

In [10]:
def top_5_accuracy(y_true, y_pred):
    y_true = y_true[:, :20]
    y_pred = y_pred[:, :20]
    return top_k_categorical_accuracy(y_true, y_pred)

In [11]:
def categorical_crossentropy(y_true, y_pred):
    y_true = y_true[:, :20]
    y_pred = y_pred[:, :20]
    return logloss(y_true, y_pred)

In [12]:
# logloss with only soft probabilities and targets
def soft_logloss(y_true, y_pred):     
    logits = y_true[:, 20:]
    y_soft = K.softmax(logits/temperature)
    y_pred_soft = y_pred[:, 20:]    
    return logloss(y_soft, y_pred_soft)

In [13]:
lambda_const = 0.07

model.compile(loss=lambda y_true, y_pred: knowledge_distillation_loss(y_true, y_pred, lambda_const),
    optimizer=optimizers.SGD(lr=1e-3),
    metrics=[categorical_crossentropy, accuracy, top_5_accuracy, soft_logloss])

In [14]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=150,
    epochs=30,
    validation_data=validation_generator,
    validation_steps=30)

Epoch 1/30
150/150 [==============================] - 23s 153ms/step - loss: 3.2054 - categorical_crossentropy: 2.9959 - accuracy: 0.0673 - top_5_accuracy: 0.2493 - soft_logloss: 2.9957 - val_loss: 3.2053 - val_categorical_crossentropy: 2.9950 - val_accuracy: 0.0767 - val_top_5_accuracy: 0.2633 - val_soft_logloss: 2.9957
Epoch 2/30
150/150 [==============================] - 20s 133ms/step - loss: 3.2054 - categorical_crossentropy: 2.9956 - accuracy: 0.0693 - top_5_accuracy: 0.2560 - soft_logloss: 2.9957 - val_loss: 3.2055 - val_categorical_crossentropy: 2.9948 - val_accuracy: 0.0833 - val_top_5_accuracy: 0.2850 - val_soft_logloss: 2.9957
Epoch 3/30
150/150 [==============================] - 24s 160ms/step - loss: 3.2054 - categorical_crossentropy: 2.9955 - accuracy: 0.0693 - top_5_accuracy: 0.2613 - soft_logloss: 2.9957 - val_loss: 3.2053 - val_categorical_crossentropy: 2.9946 - val_accuracy: 0.0817 - val_top_5_accuracy: 0.2817 - val_soft_logloss: 2.9957
Epoch 4/30
150/150 [===========

150/150 [==============================] - 29s 193ms/step - loss: 3.2051 - categorical_crossentropy: 2.9914 - accuracy: 0.0930 - top_5_accuracy: 0.3457 - soft_logloss: 2.9957 - val_loss: 3.2052 - val_categorical_crossentropy: 2.9906 - val_accuracy: 0.0933 - val_top_5_accuracy: 0.3317 - val_soft_logloss: 2.9957
Epoch 27/30
150/150 [==============================] - 29s 193ms/step - loss: 3.2051 - categorical_crossentropy: 2.9912 - accuracy: 0.0920 - top_5_accuracy: 0.3493 - soft_logloss: 2.9957 - val_loss: 3.2053 - val_categorical_crossentropy: 2.9912 - val_accuracy: 0.0883 - val_top_5_accuracy: 0.3100 - val_soft_logloss: 2.9957
Epoch 28/30
150/150 [==============================] - 29s 194ms/step - loss: 3.2051 - categorical_crossentropy: 2.9909 - accuracy: 0.0933 - top_5_accuracy: 0.3490 - soft_logloss: 2.9957 - val_loss: 3.2051 - val_categorical_crossentropy: 2.9902 - val_accuracy: 0.1017 - val_top_5_accuracy: 0.3317 - val_soft_logloss: 2.9957
Epoch 29/30
150/150 [===================